In [112]:
from sys import argv
from collections import defaultdict
from nltk.tokenize import mwe
from nltk import sent_tokenize
import re
import nltk
#from pudb import set_trace
argv=[]
argv.append('')
argv.append('sound1.tex')

In [113]:
class Trie(dict):
    #https://github.com/nltk/nltk/blob/develop/nltk/collections.py
    """A Trie implementation for strings"""

    LEAF = True

    def __init__(self, strings=None):
        """Builds a Trie object, which is built around a ``dict``

        If ``strings`` is provided, it will add the ``strings``, which
        consist of a ``list`` of ``strings``, to the Trie.
        Otherwise, it'll construct an empty Trie.

        :param strings: List of strings to insert into the trie
            (Default is ``None``)
        :type strings: list(str)

        """
        super(Trie, self).__init__()
        if strings:
            for string in strings:
                self.insert(string)

    def insert(self, string):
        if len(string):
            self[string[0]].insert(string[1:])
        else:
            if not self[Trie.LEAF]:
                self[Trie.LEAF] = 1
            else:
                self[Trie.LEAF]+=1


    def __missing__(self, key):
        self[key] = Trie()
        return self[key]


In [114]:
def add_new_token(string):
    tokenizer.add_mwe(r"{}".format(string))


def manual_entries(pattern, data):
    resp = re.findall(pattern, data)
    if resp:
        for match in resp:
            add_new_token(match)


def print_symbols_not_used():
    for k, v in symbols_use_count.items():
        if v == 0:
            print(k)


def read_file(path, f_name):
    with open("{}/{}".format(path, f_name), "r", encoding="ISO-8859-1") as f:
        data = f.read()
    return data


def symbols_not_accounted_for(seen_count):
    for item in sorted(dct.items(), key=lambda x: -x[1]):
        if item[1] > seen_count:
            print(item[0])

#will be used to expand simple new commands when tokens are not recognized.
def simple_replace(s):
    #works, needs more testing
    match = [x for x in re.findall(r"\\newcommand{(.*?)}{(.*?)}",s)]
    commands = {}
    if match:
        for tpl in match:
            commands[tpl[0]] = lambda x: x.replace(tpl[0],tpl[1])
    return commands

'''
for newcmd in re.findall(r'\\newcommand.*',data):
    dct.update(simple_replace(newcmd))
'''
""" \\newcommand{\\ri}{\\right} -> dct['\\ri']('hello \\ri') -> 'hello \\right' """


symbols = list()
dct = defaultdict(int)
tokenizer = mwe.MWETokenizer(separator="")
symbols_use_count = defaultdict(int)

path = "."
data = [x.replace("\n", "") for x in read_file(path, "latex_math.txt").splitlines()]
symbols.extend(data)

#for now only single expressions
path = "."
data = [x.replace("\n", "") for x in read_file(path, "user_defined_symbols_expressions.txt").splitlines()]
symbols.extend(data)


data = [x.replace("\n", "") for x in read_file(path, "latex_misc.txt").splitlines()]
symbols.extend(data)

data = [x.replace("\n", "") for x in read_file(path, "english_vocab.txt").splitlines()]
symbols.extend(data)
symbols = list(set(symbols))
symbols = sorted(symbols, key=lambda x: -len(x))
for symbol in symbols:
    add_new_token(symbol)

file_data = read_file(".", argv[1])
def use_package(s):
    import re
    match = re.match(r"(?P<cmd>\\usepackage)(?P<package>{[a-z]+(,[a-z]+)*})?", s)
    if match:
        cmd = match.group("cmd")
        packages = match.group("package")
        if packages:
            packages = [x.strip() for x in re.split("{|}|,", packages) if x.strip()]
            if packages:
                return {cmd: packages}


def math_mode(s):
    import re
    match = re.findall(r"[^\$](\$\$.*?\$\$)[^\$]", s, re.DOTALL)
    single_line = re.findall(r"[^\$](\$.*?\$)[^\$]", s)
    match.extend(single_line)
    return match


def balanced(start, s, left_symbol=r"{", right_symbol=r"}"):
    import re
    matched = []
    # the pattern should include one instance of the left symbol so that balanced=-1
    # if left == right then +- counting fails
    # begin syntax handled automatically, so there is no need to include left_right symbols
    s = re.sub(r'\\newcommand.*','',file_data)
    if "\\begin" in start:
        match = re.findall(r"\\begin{(.*?)}", start)
        if match:
            match = match[0]
            # print(match)
            left_symbol = f"""\\begin{{{match}}}"""
            right_symbol = f"""\\end{{{match}}}"""
    print(left_symbol)
    if left_symbol != right_symbol:
        current_offset = 0
        balanced = 0
        match = re.finditer(r"{}".format(start), s)
        # print(match)
        for m in match:
            if m:
                # print(m)
                start_offset = m.start()
                current_offset = m.end() + 1
                balanced = -1
                while balanced != 0:
                    if (
                        s[current_offset : current_offset + len(right_symbol)]
                        == right_symbol
                    ):
                        balanced += 1
                    if (
                        s[current_offset : current_offset + len(left_symbol)]
                        == left_symbol
                    ):
                        print("-1")
                        balanced -= 1
                    current_offset += 1
                matched.append(s[start_offset:current_offset])
                start_offset = current_offset
    return matched
#balanced(r"\\author{", file_data)
#balanced(r"\\cite{", file_data)
#balanced(r"\\begin{abstract}", file_data)
#print()
#print(r"\usepackage{amsmath,amsfonts,amssymb,latexsym,cite}")
#packages = use_package(r"\usepackage{amsmath,amsfonts,amssymb,latexsym,cite}")
#print(packages)
results=[]
resp =balanced(r"\\begin{array}",file_data)
if resp:
    results.extend(resp)
resp = balanced(r"\\begin{eqnarray}",file_data)
'''if resp:
    results.extend(resp)
resp = balanced(r"\\begin{eqnarray*}",file_data)
if resp:
    results.extend(resp)


resp = balanced(r"\\begin{align}",file_data)
if resp:
    results.extend(resp)

resp = balanced(r"\\begin{alignat}",file_data)
if resp:
    results.extend(resp)

resp = balanced(r"\\begin{aligned}",file_data)
if resp:
    results.extend(resp)
resp = balanced(r"\\begin{alignedat}",file_data)
if resp:
    results.extend(resp)
resp = balanced(r"\\begin{Bmatrix}",file_data)
if resp:
    results.extend(resp)
resp = balanced(r"\\begin{bmatrix}",file_data)
if resp:
    results.extend(resp)
resp = balanced(r"\\begin{gather}",file_data)
if resp:
    results.extend(resp)
resp = balanced(r"\\begin{gathered}",file_data)
if resp:
    results.extend(resp)
resp = balanced(r"\\begin{matrix}",file_data)
if resp:
    results.extend(resp)
resp = balanced(r"\\begin{multline}",file_data)
if resp:
    results.extend(resp)
resp = balanced(r"\\begin{pmatrix}",file_data)
if resp:
    results.extend(resp)
resp = balanced(r"\\begin{smallmatrix}",file_data)
if resp:
    results.extend(resp)
resp = balanced(r"\\begin{split}",file_data)
if resp:
    results.extend(resp)
resp = balanced(r"\\begin{subarray}",file_data)
if resp:
    results.extend(resp)
resp = balanced(r"\\begin{Vmatrix}",file_data)
if resp:
    results.extend(resp)
resp = balanced(r"\\begin{vmatrix}",file_data)
if resp:
    results.extend(resp)
'''
dct = Trie()
math_tex = math_mode(file_data)
math_tex.extend(results)

for item in math_tex:
    print(item)
    resp = tokenizer.tokenize(item)
    if resp[0] == '$' and resp[-1] == '$':
        resp = resp[1:-1]
        #dct.insert(resp)
        print(resp)
        print('\n\n')

print(dct)


\begin{array}
\begin{eqnarray}
$\alpha$
['\\alpha']



$\frac{m_p}{m_e}$
['\\frac', '{', 'm', '_', 'p', '}', '{', 'm', '_', 'e', '}']



$v_u$
['v', '_', 'u']



$\frac{v_u}{c}=\alpha\left(\frac{m_e}{2m_p}\right)^{\frac{1}{2}}$
['\\frac', '{', 'v', '_', 'u', '}', '{', 'c', '}', '=', '\\alpha', '\\left', '(', '\\frac', '{', 'm', '_', 'e', '}', '{', '2', 'm', '_', 'p', '}', '\\right', ')', '^', '{', '\\frac{1}{2}', '}']



$c$
['c']



$m_e$
['m', '_', 'e']



$e$
['e']



$\hbar$
['\\hbar']



$\alpha$
['\\alpha']



$\frac{m_p}{m_e}$
['\\frac', '{', 'm', '_', 'p', '}', '{', 'm', '_', 'e', '}']



$\alpha$
['\\alpha']



$\frac{m_p}{m_e}$
['\\frac', '{', 'm', '_', 'p', '}', '{', 'm', '_', 'e', '}']



$\alpha$
['\\alpha']



$\alpha$
['\\alpha']



$\frac{m_p}{m_e}$
['\\frac', '{', 'm', '_', 'p', '}', '{', 'm', '_', 'e', '}']



$v$
['v']



$v$
['v']



$v_u$
['v', '_', 'u']



$c$
['c']



$v$
['v']



$v$
['v']



$v$
['v']



$v$
['v']



$v$
['v']



$v$
['v']



$v=\left({\frac{M}

In [115]:
argv[1]

'sound1.tex'

In [116]:
pwd

'/home/user/latex-in-arxiv'

In [118]:
data

['proton-to-electron',
 'order-of-magnitude',
 'electron-to-proton',
 'circumnavigation',
 'circumnavigating',
 'acknowledgements',
 'trigonometrical',
 'transplantation',
 'transformations',
 'thermodynamical',
 'supersymmetries',
 'supernumeraries',
 'superconductors',
 'superconducting',
 'strongly-bonded',
 'straightforward',
 'simplifications',
 'representatives',
 'representations',
 'renormalization',
 'regularizations',
 'recommendations',
 'proportionality',
 'parametrization',
 'observationally',
 'nucleosynthesis',
 'notwithstanding',
 'nonrelativistic',
 'nonperturbative',
 'nonconservation',
 'multiplications',
 'microscopically',
 'logarithmically',
 'irreversibility',
 'interpretations',
 'inter-dependent',
 'interchangeably',
 'inhomogeneities',
 'infinitesimally',
 'inconsistencies',
 'inconsequential',
 'incompatibility',
 'identifications',
 'holographically',
 'gravitationally',
 'generalizations',
 'generalisations',
 'electromagnetic',
 'electrodynamics',
 'distin

In [119]:
import nltk

In [120]:
sents = nltk.sent_tokenize(file_data)

In [121]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [130]:
from time import sleep
from collections import defaultdict
data_dict = defaultdict(list)
titles = balanced(r"\\title{",file_data)
if titles:
    for title in titles:
        data_dict['title'].append(title)
        file_data = re.sub(re.escape(title),'',file_data)
        
authors = balanced(r"\\author{",file_data)
if authors:
    for author in authors:
        data_dict['author'].append(author)
        file_data = re.sub(re.escape(author),'',file_data)
        
affiliations = balanced(r"\\affiliation{",file_data)
if affiliations:
    for affiliation in affiliations:
        data_dict['affiliation'].append(affiliation)
        file_data = re.sub(re.escape(affiliation),'',file_data)
        
equations = balanced(r"\\begin{equation}",file_data)
if equations:
    data_dict['equation'].append(resp)
    for equation in equations:
        file_data = re.sub(re.escape(equation),'',file_data,re.MULTILINE)

{
{
{
\begin{equation}


In [132]:
label='\\being{figure}'
code_gen=f'''{label}s = balanced(r"\\{label}",file_data)
if {label}s:
    for {label} in {label}s:
        data_dict['{label}'].append({label})
        file_data = re.sub(re.escape({label}),'',file_data)'''

In [133]:
print(code_gen.replace('\\','\\\\'))

\\being{figure}s = balanced(r"\\\\being{figure}{",file_data)
if \\being{figure}s:
    for \\being{figure} in \\being{figure}s:
        data_dict['\\being{figure}'].append(\\being{figure})
        file_data = re.sub(re.escape(\\being{figure}),'',file_data)


In [138]:
figures = balanced(r"\\begin{figure}",file_data)
if figures:
    for figure in figures:
        data_dict['figure'].append(figure)
        file_data = re.sub(re.escape(figure),'',file_data)

\begin{figure}


In [140]:
figure

'\\begin{figure}\n\\caption{Calculated speed of sound in atomic hydrogen as a function of pressure (top) and density (bottom). The dashed line shows the upper bound $v_u$ in (\\ref{v3}).}\n\\label{abinitio}\n\\'

In [93]:
for ix,line in enumerate(file_data.splitlines()):
    #fd = line
    if '\\usepackage' in line:
        resp = use_package(line)
        if resp:
            for items in resp.values():
                for item in items:
                    data_dict['usepackage'].append(item)
            continue
    if '\\documentclass\[' in line:
        data_dict['documentclass'].append(line)
        continue
        
    if '%' in line:
        resp = False
        resp = re.findall('%.*',line)
        if resp:
            data_dict['comment'].append(line)
        continue
        
    
    else:
        try:
            line = re.sub(r'\\begin{document}|\\raggedbottom|\\maketitle|\\section{.*?}','',line)
            
        except Exception as e:
            print(e)
    sleep(5)

KeyboardInterrupt: 

{


['\\title{Speed of sound from fundamental physical constants}']